<a href="https://colab.research.google.com/github/nkashanov/my-project/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [4]:
 import pandas as pd

In [2]:
!git clone https://github.com/nkashanov/my-project.git

Cloning into 'my-project'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 61 (delta 0), reused 2 (delta 0), pack-reused 57 (from 1)
Receiving objects: 100% (61/61), 43.80 MiB | 15.29 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Updating files: 100% (18/18), done.


In [3]:
%cd my-project/brazilian_ecommerce_analysis

/content/my-project/brazilian_ecommerce_analysis


olist_customers_dataset.csv – данные о клиентах (ID, город, штат).

olist_orders_dataset.csv – информация о заказах (дата покупки, статус).

olist_order_items_dataset.csv – данные о товарах в заказе (цена, количество).

olist_order_payments_dataset.csv – данные об оплате (способы оплаты, суммы).

olist_order_reviews_dataset.csv – отзывы клиентов (оценки, комментарии).

olist_products_dataset.csv – информация о товарах (категория, размеры).

olist_sellers_dataset.csv – информация о продавцах (местоположение, ID).

olist_geolocation_dataset.csv – геоданные (широта/долгота городов).

product_category_name_translation.csv – перевод категорий товаров.

# START

In [16]:
# Загружаем ключевые датасеты
customers = pd.read_csv("olist_customers_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")

In [18]:
#проверяем информацию о данных
print(customers.info())
print(orders.info())
print(order_items.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  obje

In [22]:
#смотрим пропуски
print(customers.isnull().sum())
print(orders.isnull().sum())
print(order_items.isnull().sum())

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


In [26]:
#Отобразим order_status для заказов, где order_delivered_customer_date пустая
print(orders[orders["order_delivered_customer_date"].isnull()]["order_status"].value_counts())

order_status
shipped       1107
invoiced       314
processing     301
delivered        8
created          5
approved         2
Name: count, dtype: int64


In [25]:
#удаляем отмененные и недоступные заказы
orders = orders[~orders["order_status"].isin(["canceled", "unavailable"])]

In [28]:
#заказы delivered, но без даты (order_delivered_customer_date) – возможно, ошибка
#заполним order_delivered_customer_date значением order_estimated_delivery_date, если оно есть

orders.loc[(orders["order_status"] == "delivered") &
           (orders["order_delivered_customer_date"].isnull()),
           "order_delivered_customer_date"] = orders["order_estimated_delivery_date"]

In [33]:
orders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [35]:
#преобразуем даты в datetime
date_columns = [
    "order_purchase_timestamp", "order_approved_at",
    "order_delivered_carrier_date", "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_columns:
    orders[col] = pd.to_datetime(orders[col])

In [36]:
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


In [37]:
#начнем с даты подтверждения заказа order_approved_at
#если ее нет то заказ неодобрен - удаляем
orders = orders.dropna(subset=["order_approved_at"])

In [39]:
#проверка
print(orders["order_approved_at"].isnull().sum())  # Должно быть 0
print(orders.shape)  # Посмотрим, сколько строк осталось

0
(98188, 8)


In [40]:
#есть ли связь между пропусками и статусом заказа?
print(orders[orders["order_delivered_customer_date"].isnull()]["order_status"].value_counts())

order_status
shipped       1107
invoiced       314
processing     301
approved         2
Name: count, dtype: int64


In [41]:
orders["order_delivered_customer_date"].fillna("not_delivered", inplace=True)

<ipython-input-41-aa1bb6a65c3b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  orders["order_delivered_customer_date"].fillna("not_delivered", inplace=True)
<ipython-input-41-aa1bb6a65c3b>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'not_delivered' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  orders["order_delivered_customer_date"].fillna("not_delivered", inplace=True)
<ipython-input-41-

In [42]:
#разобраться со вставкой: заменить пропущенные даты на "не доставлено" и с предупреждением